In [1]:
# IMPORTS
import GDI
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

/Users/joe/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# PARAMETERS

# number of nodes
R  = 11

# number of samples
N  = 1e5  

# memory parameter
M  = 3     

# bootstrap iterations
B = 2

# weights/betas
p1 = 0.5
p2 = 0.75
p3 = 0.75
p4 = 0.5
p5 = 0.75

P = 2

In [3]:
# GENERATE RAW DATA
# X has dim: NxR
X = np.random.uniform(size=(int(N),int(R)))
Z = np.random.multivariate_normal(np.zeros((R,)), np.eye(R), int(N))

In [4]:
# IMPLEMENT CAUSALITY/STRUCTURE
# shift things in time to create causal Gaussian network with weights used before
X[1:,0]  = np.sqrt(1-p1)*Z[1:,0]  + np.sqrt(p1)*(X[:-1,5]**P)
X[1:,2]  = np.sqrt(1-p1)*Z[1:,2]  + np.sqrt(p1)*(X[:-1,5]**P)
X[1:,4]  = np.sqrt(1-p1)*Z[1:,4]  - np.sqrt(p1)*(X[:-1,5]**P)
X[1:,3]  = np.sqrt(1-p2)*Z[1:,3]  + np.sqrt(p2)*(X[:-1,0]**P)
X[1:,1]  = np.sqrt(1-p3)*Z[1:,1]  + np.sqrt(p3/4)*(X[:-1,6]**P) - np.sqrt(p3/4)*(X[:-1,7]**P) + np.sqrt(p3/4)*(X[:-1,8]**P) + np.sqrt(p3/4)*(X[:-1,5]**P)
X[1:,10] = np.sqrt(1-p4)*Z[1:,10] + np.sqrt(p4)*(X[:-1,3]**P)
X[1:,9]  = np.sqrt(1-p5)*Z[1:, 9] + np.sqrt(p5/2)*(X[:-1,4]**P) + np.sqrt(p5/2)*(X[:-1,2]**P)

In [5]:
# TRUE CONNECTIVITY
true_connectivity = np.zeros((R,R))
true_connectivity[5,[0, 2, 4,]] = np.asarray([p1, p1, -p1,])
true_connectivity[[5, 6, 8,],1] = p3/4
true_connectivity[7,1] = -p3/4
true_connectivity[[2, 4],9] = p5/2
true_connectivity[0,3] = p2
true_connectivity[3,10] = p4

In [ ]:
# ESTIMATE DI+GDI
X_GDI = GDI.GDI(X,M,B)
X_DI  = GDI.DI(X,M,B)

Tester = Classifier, metric = donsker_varadhan


W0928 15:58:37.426656 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/CCMI.py:79: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.



Tester = Classifier, metric = donsker_varadhan


W0928 15:58:37.465864 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/Classifier_MI.py:66: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0928 15:58:37.478031 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/CCMI.py:79: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0928 15:58:37.497778 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/Classifier_MI.py:49: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.
W0928 15:58:37.506844 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/Classifier_MI.py:66: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.




Tester = Classifier, metric = donsker_varadhan


...

W0928 15:58:37.709172 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/CCMI.py:79: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.
W0928 15:58:37.733453 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/Classifier_MI.py:49: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.
W0928 15:58:37.731380 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/CCMI.py:79: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.





Tester = Classifier, metric = donsker_varadhan


W0928 15:58:37.763794 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/Classifier_MI.py:66: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.
W0928 15:58:37.757818 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/CCMI.py:79: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.


W0928 15:58:37.782814 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/Classifier_MI.py:66: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.
W0928 15:58:37.785872 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/Classifier_MI.py:49: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.


W0928 15:58:37.802511 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/Classifier_MI.py:49: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.
W0


W0928 15:58:39.074723 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/Classifier_MI.py:84: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0928 15:58:39.087635 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/Classifier_MI.py:87: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0928 15:58:39.099137 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/Classifier_MI.py:84: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.
W0928 15:58:39.099560 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/Classifier_MI.py:84: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.


W0928 15:58:39.106042 4723125696 deprecation_wrapper.py:119] From /Users/joe/GDI_Code_Dev/Classifier_MI.py:87: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.
W0928 15:58:39.106469 4723125696 dep

Iteraion = 5000, Test accuracy = 0.736021101474762
Iteraion = 5000, Test accuracy = 0.6216702461242676
Iteraion = 5000, Test accuracy = 0.7464602589607239
Iteraion = 5000, Test accuracy = 0.615310788154602
Iteraion = 5000, Test accuracy = 0.6178905963897705
Iteraion = 5000, Test accuracy = 0.6673266291618347
Iteraion = 5000, Test accuracy = 0.737401008605957
Iteraion = 5000, Test accuracy = 0.6783057451248169
Iteraion = 5000, Test accuracy = 0.6814854741096497
Iteraion = 5000, Test accuracy = 0.7375209927558899
Iteraion = 5000, Test accuracy = 0.6241300702095032
Iteraion = 5000, Test accuracy = 0.6112310886383057
Iteraion = 5000, Test accuracy = 0.7424405813217163
Iteraion = 5000, Test accuracy = 0.7440004944801331
Iteraion = 5000, Test accuracy = 0.6153707504272461
Iteraion = 5000, Test accuracy = 0.6803455948829651
Iteraion = 5000, Test accuracy = 0.6838253140449524
Iteraion = 5000, Test accuracy = 0.739680826663971
Iteraion = 5000, Test accuracy = 0.6190304756164551
Iteraion = 5000,

In [ ]:
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the correct aspect ratio
sns.heatmap(true_connectivity, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.title('Connectivity')

In [ ]:
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Draw the heatmap with the correct aspect ratio
sns.heatmap(X_DI, cmap=cmap, vmax=.5, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.title('Estimated DI')

In [ ]:
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Draw the heatmap with the correct aspect ratio
sns.heatmap(X_GDI, cmap=cmap, vmax=0.5, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.title('Estimated GDI')